In [1]:
import time
import numpy as np
import pandas as pd
from functions import *

In [2]:
split_data_tp1_app = open_file('../data/data_tp1_app.txt')
split_data_tp1_dec = open_file('../data/data_tp1_dec.txt')

split_data_tp2_app = open_file('../data/data_tp2_app.txt')
split_data_tp2_dec = open_file('../data/data_tp2_dec.txt')

split_data_tp3_app = open_file('../data/data_tp3_app.txt')
split_data_tp3_dec = open_file('../data/data_tp3_dec.txt')

In [39]:
def create_hyperplans_classes(train_data):
    hyper_plans_classes = []
    classes = get_unique_class_num(train_data)
    for i in range(len(classes)):
        for j in range(i+1, len(classes)):
            hyper_plans_classes.append((classes[i], classes[j]))
    return hyper_plans_classes

def predict(weights, inputs):
    weight_sum = np.dot(inputs, weights[1:]) + weights[0]
    if weight_sum > 0:
        prediction = 1
    else:
        prediction = 0            
    return prediction

def linear_perceptron_two_classes(train_data, learning_rate = 0.2, n = 100):
    weights = np.zeros(len(train_data.columns))
    for _ in range(n):
        for index, row in train_data.iterrows():
            temp_line = row.tolist()
            line = list(map(float, temp_line))
            prediction = predict(weights, line[1:])

            weights[1:] += learning_rate * (line[0] - prediction) * index
            weights[0] += learning_rate * (line[0] - prediction)
    return weights

def linear_train(train_data, learning_rate = 0.2, n = 100):
    hyper_plan_list = create_hyperplans_classes(train_data)
    data_pd = pd.DataFrame(train_data)
    for hyper_plan in hyper_plan_list:

        hyperplan_data = data_pd[(data_pd[0] == hyper_plan[0]) | (data_pd[0] == hyper_plan[1])]
        hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[0]), 0] = 0
        hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[1]), 0] = 1
        weights = linear_perceptron_two_classes(hyperplan_data, learning_rate, n)
        print(weights)

linear_train(split_data_tp1_app)

<ipython-input-39-2bfd281b1e30>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[0]), 0] = 0
<ipython-input-39-2bfd281b1e30>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hyperplan_data.iloc[(hyperplan_data[0] == hyper_plan[1]), 0] = 1
[-91.  29.  29.]
[1.296000e+02 2.196114e+05 2.196114e+05]
[  1069.4        654311.39999999 654311.39999999]
[4.940000e+01 3.944914e+05 3.944914e+05]
[-760.2 3614.4 3614.4]
[2.0000e+01 4.0299e+05 4.0299e+05]
[ -1000. 143170. 143170.]
[  1069.4        466607.40000001 466607.40000001]
[4.940000e+01 2.067874e+05 2.067874e+05]
[   980.